In [450]:
using CoordinateTransformations
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

true

In [472]:
include("boxatlas.jl")

Box

In [473]:
model = Box.BoxAtlas(stiffness=1.0, damping=1.0, viscous_friction=100.)

Box.BoxAtlas{Float64}
  position_limits: Dict{Box.Body,Polyhedra.SimpleHRepresentation{2,Float64}}
  velocity_limits: Dict{Box.Body,Polyhedra.SimpleHRepresentation{2,Float64}}
  effort_limits: Dict{Box.Body,Float64}
  masses: Dict{Box.Body,Float64}
  stiffness: Float64 1.0
  damping: Float64 1.0
  gravity: Float64 10.0
  viscous_friction: Float64 100.0
  Δt: Float64 0.1


In [474]:
vis = Visualizer()[:boxatlas]
setgeometry!(vis, model)

Visualizer with path prefix Symbol[:boxatlas, :environment, :right_wall] using LCM LCMCore.LCM(Ptr{Void} @0x000000000a3634f0, "", RawFD(434), LCMCore.Subscription[LCMCore.Subscription{LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#9{DrakeVisualizer.CoreVisualizer}}}(LCMCore.SubscriptionOptions{DrakeVisualizer.Comms.CommsT,DrakeVisualizer.#handle_msg#9{DrakeVisualizer.CoreVisualizer}}(DrakeVisualizer.Comms.CommsT, DrakeVisualizer.handle_msg), Ptr{Void} @0x000000000d7b2150)])

In [475]:
state = Box.State(vcat(rand(2), randn(8), 1 * randn(10)))
input = Box.Input(zeros(8))
model.stiffness = 10.
model.damping = 10
model.gravity = 10.0
model.Δt = 0.002
for i in 1:1000
    settransform!(vis, model, state)
    model.stiffness += 10
    model.damping = 1.0 * sqrt(model.stiffness)
    state = Box.update(model, state, input)
    sleep(0.001)
end
    

In [ ]:
state.position[Box.Trunk] = [0.5, state.position[Box.Trunk][2]]
state.velocity[Box.Trunk] = [-2, 0]
# model.stiffness=2000
model.Δt = 0.05
us, xs = Box.run_mpc(model, state, 6)

In [469]:
for x in xs
    settransform!(vis, model, x)
    sleep(0.1)
end